In [10]:
import numpy as np
import pandas as pd
from datetime import datetime
import random

# Configuration de base
N_SAMPLES = 40000
random.seed(42)
np.random.seed(42)

# Statistiques de référence (HCP)
REVENU_MOYEN_GLOBAL = 21949
REVENU_MOYEN_URBAIN = 26988
REVENU_MOYEN_RURAL = 12862
PCT_URBAIN = 0.63
PCT_HOMME = 0.50

def generate_maroccan_income_dataset():
    """Génère un dataset complet des revenus marocains avec état matrimonial"""
    
    data = {}

    # ======================================
    # 1. Colonnes démographiques de base
    # ======================================
    data['id'] = np.arange(1, N_SAMPLES + 1)
    data['milieu'] = np.random.choice(['Urbain', 'Rural'], size=N_SAMPLES, p=[PCT_URBAIN, 1-PCT_URBAIN])
    data['sexe'] = np.random.choice(['Homme', 'Femme'], size=N_SAMPLES, p=[PCT_HOMME, 1-PCT_HOMME])
    
    # Âge avec distribution normale
    data['age'] = np.clip(np.random.normal(40, 12, N_SAMPLES).astype(int), 18, 90)
    data['categorie_age'] = pd.cut(data['age'], bins=[0, 25, 45, 65, 100], 
                                 labels=['Jeune', 'Adulte', 'Senior', 'Âgé'])
    
    # ======================================
    # 2. État matrimonial (nouveau)
    # ======================================
    etat_probs = {
        'Jeune': [0.75, 0.20, 0.04, 0.01],  # [Célibataire, Marié, Divorcé, Veuf]
        'Adulte': [0.20, 0.70, 0.07, 0.03],
        'Senior': [0.10, 0.60, 0.15, 0.15],
        'Âgé': [0.05, 0.40, 0.10, 0.45]
    }
    
    data['etat_matrimonial'] = [
        np.random.choice(
            ['Célibataire', 'Marié', 'Divorcé', 'Veuf'],
            p=etat_probs[cat_age]
        )
        for cat_age in data['categorie_age']
    ]
    
    # ======================================
    # 3. Éducation et emploi
    # ======================================
    education_probs = {
        'Urbain': [0.10, 0.35, 0.35, 0.20],  # Sans niveau, Fondamental, Secondaire, Supérieur
        'Rural': [0.45, 0.40, 0.12, 0.03]
    }
    
    data['niveau_education'] = [
        np.random.choice(['Sans niveau', 'Fondamental', 'Secondaire', 'Supérieur'],
                       p=education_probs[milieu])
        for milieu in data['milieu']
    ]
    
    # Expérience professionnelle (avec état matrimonial comme facteur)
    data['annees_experience'] = []
    for i in range(N_SAMPLES):
        age = data['age'][i]
        educ = data['niveau_education'][i]
        etat = data['etat_matrimonial'][i]
        
        # Âge de début de travail ajusté selon état matrimonial
        age_debut = {
            'Sans niveau': 15 if etat != 'Marié' else 17,
            'Fondamental': 18,
            'Secondaire': 21,
            'Supérieur': 24
        }[educ]
        
        exp_potentielle = max(0, age - age_debut)
        # Réduction d'expérience pour les femmes mariées en rural
        if data['sexe'][i] == 'Femme' and data['milieu'][i] == 'Rural' and etat == 'Marié':
            exp_potentielle = int(exp_potentielle * 0.7)
            
        data['annees_experience'].append(int(np.clip(np.random.normal(exp_potentielle, 3), 0, 50)))
    
    # ======================================
    # 4. Catégorie socioprofessionnelle
    # ======================================
    def determine_csp(educ, milieu, age, etat):
        # Base par éducation
        if educ == 'Supérieur':
            probs = [0.4, 0.4, 0.05, 0.0, 0.1, 0.05]  # Groupes 1 à 6
        elif educ == 'Secondaire':
            probs = [0.1, 0.4, 0.1, 0.05, 0.25, 0.1]
        elif educ == 'Fondamental':
            probs = [0.01, 0.15, 0.2, 0.2, 0.35, 0.09]
        else:  # Sans niveau
            probs = [0.0, 0.05, 0.15, 0.3, 0.3, 0.2]
        
        # Ajustements
        if milieu == 'Rural':
            probs[3] *= 2  # Plus d'agriculteurs
        if etat == 'Marié':
            probs[1] *= 1.2  # Plus de cadres moyens
        if age > 65:
            probs = [0.05, 0.05, 0.7, 0.05, 0.1, 0.05]  # Retraités
        
        return np.random.choice(['Groupe 1', 'Groupe 2', 'Groupe 3', 'Groupe 4', 'Groupe 5', 'Groupe 6'], 
                              p=np.array(probs)/sum(probs))
    
    data['categorie_socioprofessionnelle'] = [
        determine_csp(data['niveau_education'][i], data['milieu'][i], data['age'][i], data['etat_matrimonial'][i])
        for i in range(N_SAMPLES)
    ]
    
    # ======================================
    # 5. Possession de biens (ajusté par état matrimonial)
    # ======================================
    biens = ['voiture', 'logement', 'terrain']
    for bien in biens:
        data[f'possession_{bien}'] = np.zeros(N_SAMPLES, dtype=int)
    
    for i in range(N_SAMPLES):
        csp = data['categorie_socioprofessionnelle'][i]
        etat = data['etat_matrimonial'][i]
        milieu = data['milieu'][i]
        
        # Facteurs de base
        csp_factor = {
            'Groupe 1': 0.9, 'Groupe 2': 0.7, 'Groupe 3': 0.5,
            'Groupe 4': 0.3, 'Groupe 5': 0.2, 'Groupe 6': 0.1
        }[csp]
        
        # Bonus pour les mariés
        etat_factor = 1.3 if etat == 'Marié' else 1.0
        
        # Probabilités
        p_voiture = min(0.95, csp_factor * (1.2 if milieu == 'Urbain' else 0.8) * etat_factor)
        p_logement = min(0.95, csp_factor * (1.0 if milieu == 'Urbain' else 1.5) * (1.5 if etat == 'Marié' else 1.0))
        p_terrain = min(0.95, csp_factor * (0.6 if milieu == 'Urbain' else 2.0))
        
        data['possession_voiture'][i] = int(np.random.random() < p_voiture)
        data['possession_logement'][i] = int(np.random.random() < p_logement)
        data['possession_terrain'][i] = int(np.random.random() < p_terrain)
    
    # ======================================
    # 6. Nouvelles colonnes personnalisées
    # ======================================
    # a) Type de logement (synthèse)
    data['type_logement'] = [
        'Propriétaire' if poss else np.random.choice(['Locataire', 'Hébergé', 'Sans abri'], p=[0.7, 0.25, 0.05])
        for poss in data['possession_logement']
    ]
    
    # b) Accès à Internet (influence état matrimonial)
    data['acces_internet'] = [
        np.random.choice(
            ['Aucun', 'Mobile', 'Fixe'],
            p=[0.15, 0.70, 0.15] if (milieu == 'Urbain' and etat == 'Célibataire') else
              [0.50, 0.45, 0.05] if (milieu == 'Rural' and etat == 'Marié') else
              [0.25, 0.65, 0.10] if milieu == 'Urbain' else
              [0.55, 0.40, 0.05]
        )
        for milieu, etat in zip(data['milieu'], data['etat_matrimonial'])
    ]
    
    # c) Endettement mensuel (influence état matrimonial)
    data['endettement'] = [
        int(np.clip(
            np.random.normal(
                (800 if milieu == 'Urbain' else 300) * (1.5 if etat == 'Marié' else 1.0),
                (400 if logement == 'Propriétaire' else 200)
            ),
            0, 15000
        ))
        for milieu, etat, logement in zip(data['milieu'], data['etat_matrimonial'], data['type_logement'])
    ]
    
    # ======================================
    # 7. Génération du revenu (ajusté par état matrimonial)
    # ======================================
    data['revenu_annuel'] = []
    for i in range(N_SAMPLES):
        base = REVENU_MOYEN_URBAIN if data['milieu'][i] == 'Urbain' else REVENU_MOYEN_RURAL
        
        # Facteurs multiplicatifs
        factors = {
            'csp': {
                'Groupe 1': 1.8, 'Groupe 2': 1.3, 'Groupe 3': 0.9,
                'Groupe 4': 0.7, 'Groupe 5': 0.8, 'Groupe 6': 0.5
            }[data['categorie_socioprofessionnelle'][i]],
            'education': {
                'Sans niveau': 0.6, 'Fondamental': 0.8, 
                'Secondaire': 1.1, 'Supérieur': 1.5
            }[data['niveau_education'][i]],
            'etat': {
                'Célibataire': 1.0, 'Marié': 1.2,
                'Divorcé': 0.9, 'Veuf': 0.8
            }[data['etat_matrimonial'][i]],
            'logement': 1.4 if data['type_logement'][i] == 'Propriétaire' else 1.0,
            'internet': {
                'Aucun': 0.9, 'Mobile': 1.1, 'Fixe': 1.3
            }[data['acces_internet'][i]],
            'endettement': 1 - min(0.5, data['endettement'][i] / (base/12 * 0.4))
        }
        
        revenu = base * np.prod(list(factors.values()))
        data['revenu_annuel'].append(int(np.clip(np.random.normal(revenu, revenu*0.25), 1000, 500000)))
    
    # ======================================
    # 8. Post-traitement et qualité des données
    # ======================================
    df = pd.DataFrame(data)
    
    # Valeurs manquantes (5%)
    for col in ['niveau_education', 'annees_experience', 'type_logement', 'acces_internet', 'etat_matrimonial']:
        df.loc[np.random.random(N_SAMPLES) < 0.05, col] = np.nan
    
    # Colonne redondante: age_annee_naissance (redondante avec 'age')
    df['age_annee_naissance'] = datetime.now().year - df['age']
    
    # Colonne redondante: niveau_education_num (redondante avec 'niveau_education')
    education_map = {'Sans niveau': 0, 'Fondamental': 1, 'Secondaire': 2, 'Supérieur': 3}
    df['niveau_education_num'] = df['niveau_education'].map(education_map)
    # Valeurs aberrantes (1%)
    outlier_idx = np.random.choice(N_SAMPLES, size=int(N_SAMPLES*0.01), replace=False)
    df.loc[outlier_idx, 'endettement'] = np.random.randint(20000, 50000, size=len(outlier_idx))
    df.loc[outlier_idx, 'revenu_annuel'] = np.random.randint(600000, 1000000, size=len(outlier_idx))
    
    # Colonnes non pertinentes
    df['id_transaction'] = [f"TRX{random.randint(10000, 99999)}" for _ in range(N_SAMPLES)]
    df['date_creation'] = pd.to_datetime([datetime.now().strftime('%Y-%m-%d') for _ in range(N_SAMPLES)])
    df['couleur_preferee'] = np.random.choice(['Rouge', 'Bleu', 'Vert', 'Noir'], size=N_SAMPLES)
    
    # ======================================
    # 9. Vérification finale
    # ======================================
    print("=== Statistiques clés ===")
    print(f"Revenu moyen global: {df['revenu_annuel'].mean():.2f} DH")
    print(f"Endettement moyen: {df['endettement'].mean():.2f} DH/mois")
    
    print("\nRépartition état matrimonial:")
    print(df['etat_matrimonial'].value_counts(normalize=True))
    
    print("\nPossession de biens (moyenne):")
    print(df[['possession_voiture', 'possession_logement', 'possession_terrain']].mean())
    
    print("\nCorrélation état matrimonial-revenu:")
    print(df.groupby('etat_matrimonial')['revenu_annuel'].mean())
    
    df.to_csv("dataset_revenu_marocains_complet.csv", index=False)
    return df

# Génération et affichage
dataset = generate_maroccan_income_dataset()
print("\nAperçu du dataset (5 premières lignes):")
print(dataset.head(10))

=== Statistiques clés ===
Revenu moyen global: 25336.82 DH
Endettement moyen: 1152.07 DH/mois

Répartition état matrimonial:
etat_matrimonial
Marié          0.604437
Célibataire    0.232686
Divorcé        0.091676
Veuf           0.071201
Name: proportion, dtype: float64

Possession de biens (moyenne):
possession_voiture     0.504000
possession_logement    0.579025
possession_terrain     0.399475
dtype: float64

Corrélation état matrimonial-revenu:
etat_matrimonial
Célibataire    23883.522083
Divorcé        21961.681193
Marié          27146.183807
Veuf           17277.992248
Name: revenu_annuel, dtype: float64

Aperçu du dataset (5 premières lignes):
   id  milieu   sexe  age categorie_age etat_matrimonial niveau_education  \
0   1  Urbain  Homme   36        Adulte            Marié       Secondaire   
1   2   Rural  Homme   43        Adulte            Marié      Fondamental   
2   3   Rural  Homme   37        Adulte            Marié      Fondamental   
3   4  Urbain  Homme   45        A

In [12]:
df = pd.read_csv("dataset_revenu_marocains_complet.csv")
df

,id,milieu,sexe,age,categorie_age,etat_matrimonial,niveau_education,annees_experience,categorie_socioprofessionnelle,possession_voiture,...,possession_terrain,type_logement,acces_internet,endettement,revenu_annuel,age_annee_naissance,niveau_education_num,id_transaction,date_creation,couleur_preferee
0,1,Urbain,Homme,36,Adulte,Marié,Secondaire,13.0,Groupe 3,1,...,0,Propriétaire,Aucun,1236,26441,1989,2.0,TRX93810,2025-04-26,Bleu
1,2,Rural,Homme,43,Adulte,Marié,Fondamental,21.0,Groupe 2,1,...,1,Propriétaire,Aucun,34,22187,1982,1.0,TRX24592,2025-04-26,Noir
2,3,Rural,Homme,37,Adulte,Marié,Fondamental,14.0,Groupe 5,0,...,0,Locataire,Mobile,631,3470,1988,1.0,TRX13278,2025-04-26,Vert
3,4,Urbain,Homme,45,Adulte,Marié,Fondamental,24.0,Groupe 3,1,...,1,Locataire,Mobile,1434,13664,1980,1.0,TRX46048,2025-04-26,Noir
4,5,Urbain,Homme,35,Adulte,Marié,Secondaire,16.0,Groupe 2,1,...,0,Propriétaire,Aucun,1561,33328,1990,2.0,TRX42098,2025-04-26,Vert
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,39996,Rural,Homme,19,Jeune,Marié,Fondamental,0.0,Groupe 4,0,...,1,Hébergé,Mobile,542,5353,2006,1.0,TRX13578,2025-04-26,Vert
39996,39997,Rural,Homme,18,Jeune,Marié,Fondamental,1.0,Groupe 5,0,...,0,Propriétaire,Aucun,363,8394,2007,1.0,TRX36005,2025-04-26,Bleu
39997,39998,Urbain,Homme,33,Adulte,Marié,Secondaire,5.0,Groupe 2,1,...,0,Propriétaire,Mobile,831,42228,1992,2.0,TRX48019,2025-04-26,Rouge
39998,39999,Urbain,Homme,35,Adulte,Marié,Fondamental,20.0,Groupe 3,1,...,0,Propriétaire,Mobile,1409,7890,1990,1.0,TRX98690,2025-04-26,Vert
